In [1]:
#Importamos la base 

import sys
!{ sys . executable } -m pip install pandas
import pandas as pd

pd . read_excel("C:/Users/eluci/OneDrive/Escritorio/valu/tesis/Base/DataBase.xlsx")
df = pd . read_excel("C:/Users/eluci/OneDrive/Escritorio/valu/tesis/Base/DataBase.xlsx")
print(df)

    YEAR  Default       DEA_USD       DCP_%       DEA_%     DEALP_USD  \
0   1970        0  5.893191e+09   93.724092   19.387846  5.254191e+09   
1   1971        0  6.353904e+09  225.753271   18.890975  5.666904e+09   
2   1972        0  6.893803e+09  130.142426   20.157614  5.959891e+09   
3   1973        0  7.374621e+09   49.710268   13.987691  6.370708e+09   
4   1974        0  7.798526e+09   44.401414   10.729963  6.881164e+09   
5   1975        0  7.901576e+09  100.091397   14.867088  6.759796e+09   
6   1976        0  9.501611e+09   51.429015   18.563271  7.952293e+09   
7   1977        0  1.167574e+10   70.864746   20.724005  8.533272e+09   
8   1978        0  1.350766e+10   57.775386   23.403469  1.007966e+10   
9   1979        0  2.117684e+10   59.441763   30.858252  1.426584e+10   
10  1980        0  2.732246e+10  111.688421   35.815560  1.693906e+10   
11  1981        0  3.580946e+10  258.105267   46.601355  2.288813e+10   
12  1982        1  4.378700e+10  366.810490   55.25

In [2]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import plot_roc_curve
import matplotlib.pyplot as plt


def evalua_metodo(modelo, X_train, Y_train, X_test, Y_test):
    '''
    Esta función calcula la matriz de confusion, el accuracy y el AUC, ademas de graficar la curva roc y calcular el ECM. 
    Al final la función nos devuelve un data frame con los resultados de estas métricas para el modelo seleccionado en 
    base a los datos ingresados.
    Input: 
        modelo: un modelo con los parámetros ya definidos.
        X_train, Y_train, X_test, Y_test: Datos de entrenamiento y test ya divididos.
    Output:
        Matriz especificando VP, FP, VN, FN, Accuracy, AUC, ECM.
        Además imprime la matriz de confusión, el Accuracy, el AUC y grafica la curva de ROC.
    '''
    # Ajustamos el modelo
    modelo.fit(X_train, Y_train)
    
    # Realizamos predicción sobre base test
    global Y_pred 
    Y_pred = modelo.predict(X_test)
    
    # Calculamos el accuracy y matriz de confusion
    matriz_confusion = confusion_matrix(Y_test, Y_pred)
    accuracy = accuracy_score(Y_test, Y_pred)
    auc = roc_auc_score(Y_test, Y_pred)
    

    # Graficamos la curva de roc
    fpr, tpr, thresholds = roc_curve(Y_test, Y_pred)
    
    #Calculamos ECM
    ecm = mean_squared_error(Y_pred, Y_test)
    
    #Guardamos los resultados en un dataframe creado con más columnas que nos servirán para las otras funciones
    output = pd.DataFrame(columns = ["Modelo", "Penalty", "Hiperparámetro", "VP", "FP", "VN", "FN", "Accuracy", "AUC", "ECM"])
    output = output.append({"VP" : matriz_confusion[0][0], "FP": matriz_confusion[0][1], "VN": matriz_confusion[1][1], "FN": matriz_confusion[1][0], "Accuracy" : accuracy, "AUC" : auc, "ECM" : ecm}, ignore_index = True)

    return output, fpr, tpr

In [3]:
# LISTA EN VEZ DE DICCIONARIO

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from matplotlib.pyplot import boxplot

def cross_validation(X, y, nombre, modelo, penalty = "l1", hyper_list = [1, 5, 10], p = 5):
    '''
    Esta función realiza p-fold cross validation para distintos modelos, lista de hiperparámetros y data set determinado. Para eso, itera sobre una lista de hiperparámetros, crea p distintas particiones de la data y guarda los resultados en un dataframe.
    Input: 
        X: Matriz de predictores.
        y: Variable dependiente.
        modelo: Modelo a utilizar.
        penalty: Tipo de regularización ("l1": Lasso, "l2": Ridge, "elasticnet": Elastic Net).
        hyper_list: Lista de hiperparámetros a evaluar.
        p: Cantidad de particiones a realizar.
    Output:
        Dataframe por modelo e hierparámetro con el ECM promedio de las distintas particiones.
    '''
    
    #Creamos la matriz en la que vamos a guardar los outputs
    datos = pd.DataFrame(columns=["Modelo", "Particion", "Penalty", "Lambda", "ECM"])
    
    #Reseteamos el indice de las matrices a utilizar para que funcione bien el split
    X.reset_index(inplace = True, drop = True)
    y.reset_index(inplace = True, drop = True)
    
    
    #Iteramos sobre los lambdas
    for hiper in hyper_list:
        
        kf = KFold(n_splits=p, shuffle=True, random_state = 1265465)
        #Iteramos sobre las distintas particiones
        for i, (train_index, test_index) in enumerate(kf.split(X)): 
            
            #Separamos los datos en train y test
            x_train, x_test = X.reindex(train_index), X.reindex(test_index)
            y_train, y_test = y.reindex(train_index), y.reindex(test_index)
            
            #Usamos evalua_metodo para obtener el ecm y agregamos los datos al dataframe
            if nombre == "Regresión logística":
                evalua_metodo(modelo(penalty = penalty, C = 1/hiper, solver='saga', l1_ratio = 0.5,random_state=1, max_iter = 1000) , x_train , y_train, x_test, y_test)
            if nombre == "KNN":
                evalua_metodo(modelo(n_neighbors = hiper) , x_train , y_train, x_test, y_test)
            if nombre == "Arbol de decisión" or nombre  == "Boosting":
                evalua_metodo(modelo(max_depth = hiper, random_state = 1) , x_train , y_train, x_test, y_test)
            if nombre == "Support Vector Machines (SVM)":
                evalua_metodo(modelo(C = 1/hiper, random_state = 1, max_iter = -1) , x_train , y_train, x_test, y_test)
            if nombre  == "Bagging":
                evalua_metodo(modelo(n_estimators = hiper, random_state = 1) , x_train , y_train, x_test, y_test)
            if nombre == "Boosting" or nombre  == "Random Forests":
                evalua_metodo(modelo(n_estimators = hiper, random_state = 1) , x_train , y_train, x_test, y_test)
            
            ecm = mean_squared_error(Y_pred, y_test)
            datos = datos.append({"Modelo" : nombre, "Particion" : i, "Penalty" : penalty if (nombre == "Regresión logística") else "-", "Hiperparámetro": hiper ,"ECM" : ecm}, ignore_index = True)
    
    #Reseteamos el indice para arreglar un problema de formato     
    datos.reset_index(inplace = True)
    
    #Generamos el boxplot para cada penalty
    box_datos = pd.DataFrame(np.column_stack(np.split(datos["ECM"], len(hyper_list))), columns = hyper_list)
    box_datos = boxplot(box_datos, labels = hyper_list)
    plt.show()
    
    #Agrupamos el dataframe en base a modelo, penalty y lambda para calcular el ECM promedio entre las particiones
    datos = datos.groupby(["Modelo", 'Penalty', "Hiperparámetro"]).agg({'ECM':'mean'})

    return datos

In [4]:
def evalua_config(nombre, modelo, hyper_list, X, y, penalty = ["l1"], p = 5):
    '''
    Esta función utiliza la función cross_validation y elije el hiperparámetro óptimo (el que minimza ECM promedio en las distintas particiones) para cada modelo.
    Input:
        modelo: Modelo a utilizar.
        hyper_list: Lista de hiperparámetros a evaluar.
        X: Matriz de predictores.
        y: Variable dependiente.
        penalty: Lista con los tipos de regularización ("l1": Lasso, "l2": Ridge, "elasticnet": Elastic Net) a probar.
        p: Cantidad de particiones a realizar en cross validation.
    Output:
        Dataframe con un hiperparámetro óptimo y el ECM correspondiente para cada modelo.
    '''
    
    #Creamos el dataframe donde guardamos el output
    lambdas_opt = pd.DataFrame(columns=["Modelo", "Penalty", "Hiperparámetro", "ECM"])
    
    #Iteramos sobre los distintos penalties
    if nombre == "Regresión logística":
        for pen in penalty:
            #Usamos cross_validation para obtener los resultados en base al penalty
            datos2 = cross_validation(X, y, nombre, modelo, pen, hyper_list, p)
    else:
        datos2 = cross_validation(X, y, nombre, modelo, "-", hyper_list, p)
    datos2.reset_index(inplace = True)
    #Lo agregamos a la matriz de output
    lambdas_opt = lambdas_opt.append(datos2.loc[datos2["ECM"].idxmin()], ignore_index = True)
       
    return lambdas_opt

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.svm import SVC 
from sklearn.ensemble import BaggingClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.model_selection import train_test_split

def evalua_multiples_metodos(X, y, penalty = ["l1", "l2", "elasticnet"], logreg_lambda = [0.00001, 0.001, 0.1, 1, 10], k_neigh = [1, 5, 10], max_depth = [5, 10, 15], C = [0.001, 0.1, 10], n_estimators = [1, 20, 50], p = 5):
    '''
    Esta función itera sobre varios modelos (Regresión logística, Análisis discriminante lineal, KNN, Arbol de decisión, Support Vector Machines (SVM), Bagging, Random Forests, Boosting) para obtener varias métricas de performance (VP, FP, VN, FN, Accuracy, AUC, ECM) bajo distintos esquemas, y las guarda en un dataframe. Además, realiza primero validación cruzada para obtener el hiperparámetro óptimo.
    Input:
        X: Matriz de predictores.
        y: Variable dependiente.
        penalty: Lista con los tipos de regularización ("l1": Lasso, "l2": Ridge, "elasticnet": Elastic Net) a probar en cross validation
        logreg_lambda: Lista de hiperparámetros a evaluar en regresión logística.
        p: Cantidad de particiones a realizar en cross validation.
        k_neigh: Lista de valores de k a evaluar en k vecinos cercanos.
        max_depth: Lista de valores a evaluar en árboles de decisión y Boosting.
        C: Lista de valores a evaluar en SVM.
        n_estimators: Lista de valores a evaluar en Random Forest y Bagging.
    Output:
        Matriz por cada modelo con el hiperparámetro óptimo, y las métricas de performance.
    '''
    
    #Dividimos la muestra en test y train para usar en los casos que no utilizan cross_validation
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=102)
    
    #Creamos un diccionario para iterar sobre los modelos
    modelos_lista = {"Regresión logística" : LogisticRegression, "Análisis discriminante lineal" : LinearDiscriminantAnalysis, "KNN" : KNeighborsClassifier, "Arbol de decisión" : DecisionTreeClassifier, "Support Vector Machines (SVM)" : SVC, "Bagging" : BaggingClassifier, "Random Forests" : RandomForestClassifier, "Boosting" : GradientBoostingClassifier}
    
    #Creamos el dataframe de output
    output = pd.DataFrame(columns = ["Modelo", "Penalty", "Hiperparámetro", "VP", "FP", "VN", "FN", "Accuracy", "AUC", "ECM"])
    
    #Iteramos sobre los modelos
    for nombre, modelo in modelos_lista.items():
        #Si es una regresión logística, obtenemos el lambda óptimo con evalua_config y luego usamos ese lambda para obtener los resultados con evalua_metodo
        if nombre != "Análisis discriminante lineal":
            
            if nombre == "Regresión logística":
                lambdas_opt = evalua_config(nombre, modelo, logreg_lambda, X, y, penalty, p)
                output = output.append(evalua_metodo(modelo(penalty = lambdas_opt["Penalty"][0], C = 1/lambdas_opt["Hiperparámetro"][0], l1_ratio = 0.5, solver = "saga"), X_train, Y_train, X_test, Y_test)[0], ignore_index = True)
            if nombre == "KNN":
                lambdas_opt = evalua_config(nombre, modelo, k_neigh, X, y, p = p)
                output = output.append(evalua_metodo(modelo(np.int_(lambdas_opt["Hiperparámetro"][0])), X_train, Y_train, X_test, Y_test)[0], ignore_index = True)
            if nombre == "Arbol de decisión" or nombre  == "Boosting":
                lambdas_opt = evalua_config(nombre, modelo, max_depth, X, y, p = p)
                output = output.append(evalua_metodo(modelo(max_depth = lambdas_opt["Hiperparámetro"][0], random_state = 1), X_train, Y_train, X_test, Y_test)[0], ignore_index = True)
            if nombre == "Support Vector Machines (SVM)":
                lambdas_opt = evalua_config(nombre, modelo, C, X, y, p = p)
                output = output.append(evalua_metodo(modelo(1/lambdas_opt["Hiperparámetro"][0], random_state = 1), X_train, Y_train, X_test, Y_test)[0], ignore_index = True)
            if nombre  == "Random Forests" or nombre == "Bagging":
                lambdas_opt = evalua_config(nombre, modelo, n_estimators, X, y, p = p)
                output = output.append(evalua_metodo(modelo(n_estimators = np.int_(lambdas_opt["Hiperparámetro"][0]), random_state = 1), X_train, Y_train, X_test, Y_test)[0], ignore_index = True)

                
            output.loc[len(output)-1, ["Modelo", "Penalty", "Hiperparámetro"]] = lambdas_opt.at[0, "Modelo"], lambdas_opt.at[0, "Penalty"], lambdas_opt.at[0, "Hiperparámetro"]
            
        #Para el resto de los casos, simplemente usamos evalua_metodo para obtener los resultados
        else:
            output = output.append(evalua_metodo(modelo(), X_train, Y_train, X_test, Y_test)[0], ignore_index = True)
            output.loc[len(output)-1, "Modelo"] = nombre
            output.fillna("-", inplace = True)
    return output

In [6]:
# Establecemos nuestras dos variables, la dependiente (y) y la independiente (x)
default = df['Default']
X = df.drop("Default", axis = 1) #osea serian todas las demas variables

In [7]:
X

,YEAR,DEA_USD,DCP_%,DEA_%,DEALP_USD,CAB2_%GDP,SP.RUR.TOTL.ZS,SP.POP.65UP.FE.IN,SP.DYN.TFRT.IN,PA.NUS.ATLS,...,MS.MIL.XPND.CN,EN.ATM.CO2E.GF.KT,DT.TDS.DPPG.GN.ZS,DT.ODA.ALLD.KD,DT.NFL.RDBN.CD,DT.DOD.DSTC.IR.ZS,BN.CAB.XOKA.GD.ZS,FM.LBL.BMNY.IR.ZS.1,PA.NUS.FCRF,BX.KLT.DINV.CD.DT
0,1970,5.893191e+09,93.724092,19.387846,5.254191e+09,NaN,21.120,896788,3.079,3.800000e-11,...,1.690000e-02,10289.602,1.554212,-3.326000e+07,21122000,93.724092,NaN,10.653097,3.791667e-11,8.977000e+07
1,1971,6.353904e+09,225.753271,18.890975,5.666904e+09,NaN,20.679,932723,3.109,5.000000e-11,...,2.350000e-02,11030.336,1.316393,2.915000e+07,26933000,225.753271,NaN,19.716443,4.521667e-11,1.256700e+08
2,1972,6.893803e+09,130.142426,20.157614,5.959891e+09,NaN,20.257,969322,3.149,8.000000e-11,...,3.830000e-02,12478.801,1.425207,6.710000e+06,-1837000,130.142426,NaN,13.756666,5.000000e-11,7.172000e+07
3,1973,7.374621e+09,49.710268,13.987691,6.370708e+09,NaN,19.842,1006756,3.198,9.000000e-11,...,6.470000e-02,14198.624,1.286844,6.160000e+07,35080000,49.710268,NaN,7.652014,5.000000e-11,1.002500e+08
4,1974,7.798526e+09,44.401414,10.729963,6.881164e+09,NaN,19.433,1045077,3.251,9.000000e-11,...,8.130000e-02,14935.691,1.115621,6.498000e+07,35489000,44.401414,NaN,9.996345,5.000000e-11,1.792000e+07
5,1975,7.901576e+09,100.091397,14.867088,6.759796e+09,NaN,19.031,1084145,3.301,3.700000e-10,...,2.920000e-01,15735.097,1.517540,4.353000e+07,47615000,100.091397,NaN,13.701111,3.657500e-10,5.559000e+07
6,1976,9.501611e+09,51.429015,18.563271,7.952293e+09,NaN,18.634,1124768,3.339,2.000000e-09,...,2.870000e+00,16479.498,1.738069,5.074000e+07,51814000,51.429015,1.272242,4.916009,1.399833e-09,2.446800e+08
7,1977,1.167574e+10,70.864746,20.724005,8.533272e+09,NaN,18.245,1165429,3.359,5.000000e-09,...,8.390000e+00,16919.538,1.885836,5.180000e+07,50228000,70.864746,1.983058,3.319509,4.076333e-09,1.440000e+08
8,1978,1.350766e+10,57.775386,23.403469,1.007966e+10,NaN,17.861,1206363,3.359,1.220000e-08,...,2.470000e+01,16439.161,3.704828,4.351000e+07,139919000,57.775386,3.195434,2.402380,7.957500e-09,2.500000e+08
9,1979,2.117684e+10,59.441763,30.858252,1.426584e+10,NaN,17.484,1247976,3.339,2.790000e-08,...,6.660000e+01,18030.639,2.170951,6.553000e+07,68911000,59.441763,-0.740769,1.540788,1.316967e-08,2.060000e+08


In [8]:
df.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 337 columns):
 #    Column                Non-Null Count  Dtype  
---   ------                --------------  -----  
 0    YEAR                  51 non-null     int64  
 1    Default               51 non-null     int64  
 2    DEA_USD               51 non-null     float64
 3    DCP_%                 51 non-null     float64
 4    DEA_%                 51 non-null     float64
 5    DEALP_USD             51 non-null     float64
 6    CAB2_%GDP             36 non-null     float64
 7    SP.RUR.TOTL.ZS        51 non-null     float64
 8    SP.POP.65UP.FE.IN     51 non-null     int64  
 9    SP.DYN.TFRT.IN        51 non-null     float64
 10   PA.NUS.ATLS           51 non-null     float64
 11   NY.GDP.MKTP.KD.ZG     51 non-null     float64
 12   NY.GDP.DEFL.KD.ZG     51 non-null     float64
 13   NV.SRV.TOTL.KN        51 non-null     float64
 14   TG.VAL.TOTL.GD.ZS     51 non-null     float64
 15   SP.RUR

C:\Users\eluci\AppData\Local\Temp/ipykernel_7484/2728897127.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df.info(verbose=True,null_counts=True)


In [9]:
df.dropna(1)

C:\Users\eluci\AppData\Local\Temp/ipykernel_7484/4279380323.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  df.dropna(1)


,YEAR,Default,DEA_USD,DCP_%,DEA_%,DEALP_USD,SP.RUR.TOTL.ZS,SP.POP.65UP.FE.IN,SP.DYN.TFRT.IN,PA.NUS.ATLS,...,NY.GDP.MKTP.CD,NY.ADJ.DNGY.CD,NV.SRV.TOTL.CD,NV.IND.MANF.KD.ZG,NE.DAB.TOTL.ZS,DT.TDS.DPPG.GN.ZS,DT.NFL.RDBN.CD,DT.DOD.DSTC.IR.ZS,PA.NUS.FCRF,BX.KLT.DINV.CD.DT
0,1970,0,5.893191e+09,93.724092,19.387846,5.254191e+09,21.120,896788,3.079,3.800000e-11,...,3.158421e+10,6.537486e+07,1.518684e+10,-2.571708,99.134317,1.554212,21122000,93.724092,3.791667e-11,8.977000e+07
1,1971,0,6.353904e+09,225.753271,18.890975,5.666904e+09,20.679,932723,3.109,5.000000e-11,...,3.329320e+10,1.029926e+08,1.437560e+10,6.165982,100.600724,1.316393,26933000,225.753271,4.521667e-11,1.256700e+08
2,1972,0,6.893803e+09,130.142426,20.157614,5.959891e+09,20.257,969322,3.149,8.000000e-11,...,3.473300e+10,1.206953e+08,1.508575e+10,3.994579,99.640110,1.425207,-1837000,130.142426,5.000000e-11,7.172000e+07
3,1973,0,7.374621e+09,49.710268,13.987691,6.370708e+09,19.842,1006756,3.198,9.000000e-11,...,5.254400e+10,1.905117e+08,2.348456e+10,3.966241,98.096830,1.286844,35080000,49.710268,5.000000e-11,1.002500e+08
4,1974,0,7.798526e+09,44.401414,10.729963,6.881164e+09,19.433,1045077,3.251,9.000000e-11,...,7.243678e+10,9.580577e+08,3.344867e+10,5.874110,99.386439,1.115621,35489000,44.401414,5.000000e-11,1.792000e+07
5,1975,0,7.901576e+09,100.091397,14.867088,6.759796e+09,19.031,1084145,3.301,3.700000e-10,...,5.243865e+10,8.713150e+08,2.269270e+10,-2.552779,100.154831,1.517540,47615000,100.091397,3.657500e-10,5.559000e+07
6,1976,0,9.501611e+09,51.429015,18.563271,7.952293e+09,18.634,1124768,3.339,2.000000e-09,...,5.116950e+10,9.518433e+08,2.095600e+10,-3.033737,96.746207,1.738069,51814000,51.429015,1.399833e-09,2.446800e+08
7,1977,0,1.167574e+10,70.864746,20.724005,8.533272e+09,18.245,1165429,3.359,5.000000e-09,...,5.678100e+10,9.053710e+08,2.504420e+10,7.813687,97.710502,1.885836,50228000,70.864746,4.076333e-09,1.440000e+08
8,1978,0,1.350766e+10,57.775386,23.403469,1.007966e+10,17.861,1206363,3.359,1.220000e-08,...,5.808287e+10,1.045685e+09,2.694926e+10,-10.522623,97.107012,3.704828,139919000,57.775386,7.957500e-09,2.500000e+08
9,1979,0,2.117684e+10,59.441763,30.858252,1.426584e+10,17.484,1247976,3.339,2.790000e-08,...,6.925233e+10,2.798103e+09,3.336272e+10,9.944679,99.818803,2.170951,68911000,59.441763,1.316967e-08,2.060000e+08


In [10]:
evalua_multiples_metodos(X, default, ["l1", "l2"], p = 10)

C:\Users\eluci\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1317: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn("l1_ratio parameter is only used when penalty is "


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').